In [0]:
# Не трожь, оно тебя сожрет!
!pip install catboost

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from preprocessing import preprocess_customer, preprocess_reactions
from answering import submit, choose_from_proba

In [0]:
path = '/content/'

cust = pd.read_csv(path + 'customer_test.csv')
transactions = pd.read_csv(path + 'transactions.csv')
react_train = pd.read_csv(path + 'stories_reaction_train.csv')
react_test = pd.read_csv(path + 'stories_reaction_test.csv')
descr = pd.read_csv(path + 'stories_description.csv')
sample = pd.read_csv(path + 'sample_submit.csv')

In [0]:
new_cust = preprocess_customer(cust, transactions, react_train, encodings=['mean', 'frequency', 'std'])
train, test = preprocess_reactions(react_train, react_test, encodings = ['mean', 'frequency', 'std'])

train = train.join(new_cust.set_index('customer_id'), on = 'customer_id')
test = test.join(new_cust.set_index('customer_id'), on = 'customer_id')

like, view, skip, dislike =  [1 3 2 0]


In [0]:
from sklearn.model_selection import train_test_split

y = train['event']
train.drop('event', axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_test_split(train, y)

In [0]:
import catboost

train_pool = catboost.Pool(data=X_train.values, label=y_train.values)
val_pool = catboost.Pool(data=X_val.values, label=y_val.values)

In [0]:
model = catboost.CatBoostClassifier(
    loss_function='MultiClass',
    task_type='GPU',
    custom_loss=['Accuracy']
)

model.fit(train_pool, plot=True)

In [99]:
y_val_pred = model.predict(X_val)
accuracy_score(y_val, y_val_pred)

0.7095514262042846

In [0]:
pred = choose_from_proba(model, test)
submit(pred, sample, 'somename.csv')